In [1]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [2]:
def RNet(X_train, y_train, X_test, y_test , hiddenlayer, node, Ouputnode, epochs):
    
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 신경망 생성하기
    model = RegressionNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
            
    #학습되 모델 저장
    torch.save(model, "RNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    pred_y_test_tensor = pred_y_test['Predarray']
   
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
    MSE = mean_squared_error(y_test, pred_y_test)  # MSE 값은 출력하지 않아도 됩니다. RMSE를 구하기 위한 절차입니다.
    
    # 출력 대상입니다. 성능지표로써 3가지값이 출력되게 됩니다.
    RMSE = np.sqrt(MSE)
    MAPE = MAPE_regression(y_test, pred_y_test)
    R_squared = r2_score(y_test, pred_y_test)
    return {'pred_y' : pred_y_test, 'RMSE' : RMSE, 'MAPE' : MAPE, "R_squared" : R_squared}

In [3]:
def MAPE_regression(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [21]:
#NuralNet (Regression)
class RegressionNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
            for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        ordered_dict['']
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        
        return {"Predarray" :pred_x}

    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))
  

IndentationError: unexpected indent (1849939986.py, line 11)

In [90]:
#NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        ordered_dict['Sigmoid']= nn.Sigmoid()
        self.layer1  = nn.Sequential(ordered_dict)
        
    def predict(self, X_test):
        Score = self.layer1(X_test)
        return {"Score" :Score}

    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [111]:

def CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs,decision_boundary=0.5):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    #신경망 생성하기
    model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "CNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    Score = model.predict(X_test)    
    
    #파이토치 텐서를 넘파이 어레이로 전환
    anomalyScore= Score['Score']
    anomalyScore = anomalyScore.detach().numpy()
    print(anomalyScore)
    anomalyScore = anomalyScore.reshape(-1)
    print(anomalyScore)
    round_anomalyScore = np.round(anomalyScore,3)
    
    pred = np.array([], dtype=np.int32)
        
    for i in range(len(round_anomalyScore)):
        if (round_anomalyScore[i] > decision_boundary) == True:
            pred = np.append(pred, 1)
        else:
            pred = np.append(pred, 0)
    # calculate AUC of model
    fpr, tpr, thresholds = roc_curve(y_test, round_anomalyScore)
    auc = roc_auc_score(y_test, round_anomalyScore)
    ###
    return  {"auc_score": auc, "fpr":fpr, "tpr":tpr, "Score": round_anomalyScore ,  "Predarray" : pred }
    

In [112]:
# ANN Model load

def ANN_model_loader(pickleFile, tsdat) :
    """
    저장한 모델을 로드한 후, 로드한 모델과 데이터를 활용해 분석 결과 리턴

    Parameters
    ----------
    model : ?
        로드한 모델
    tsdat : array
        예측 데이터

    Returns
    -------
    모델 리턴과 동일

    """
    model = torch.load(pickleFile)  
    X = torch.Tensor(tsdat.values)
    pred = model.predict(X)
    pred = pred['Predarray']
    pred = pred.detach().numpy()
    
    
    return  pred 

In [113]:
df1 = pd.read_csv("ab_te.csv")
df2 = pd.read_csv("RMS_bearing.csv")

In [114]:
x = df1.iloc[:,[0, 1, 2, 3]]
y = df1.iloc[:,[4]]
x1 = df2.iloc[:,[0,1,2]]
y1 = df2.iloc[:,[3]]


X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

X1_train,X1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=33)

In [115]:
pred_y2_test= CNet(X_train, y_train, X_test,y_test, 2, 5, 1, 10)

#CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs)

0epoch // loss =-0.0
[[0.4577924 ]
 [0.4768075 ]
 [0.4852541 ]
 [0.49406576]
 [0.47754592]
 [0.47223654]
 [0.47322214]
 [0.48546198]
 [0.5184511 ]
 [0.48367038]
 [0.47409266]
 [0.5134637 ]
 [0.5058104 ]
 [0.48272645]
 [0.47943088]
 [0.49703714]
 [0.5003481 ]
 [0.48717156]
 [0.47860688]
 [0.49560556]
 [0.46097463]
 [0.47876576]
 [0.49270892]
 [0.46189737]
 [0.5053385 ]
 [0.49149323]
 [0.50191015]
 [0.49164405]
 [0.50424653]
 [0.4872218 ]
 [0.47167805]
 [0.5046935 ]
 [0.4829589 ]
 [0.47855031]
 [0.50510204]
 [0.48179594]
 [0.4877076 ]
 [0.5113992 ]
 [0.48328668]
 [0.4763601 ]
 [0.5159488 ]
 [0.49873388]
 [0.47148848]
 [0.4819413 ]
 [0.51521015]
 [0.4897581 ]
 [0.48047757]
 [0.49411958]
 [0.49503782]
 [0.48003572]
 [0.4771273 ]
 [0.48775384]
 [0.4834432 ]
 [0.48989573]
 [0.47428983]
 [0.47683966]
 [0.48703024]
 [0.48739296]
 [0.5026392 ]
 [0.49426812]
 [0.49261126]
 [0.47191197]
 [0.45741078]
 [0.49005142]
 [0.4787982 ]
 [0.48347914]
 [0.5102711 ]
 [0.47459483]
 [0.49202055]
 [0.48195028]

In [116]:
print(pred_y2_test)

{'auc_score': 0.2892470130591831, 'fpr': array([0.        , 0.00819672, 0.02459016, 0.02459016, 0.03278689,
       0.04918033, 0.07377049, 0.07377049, 0.09836066, 0.1147541 ,
       0.13934426, 0.16393443, 0.18032787, 0.19672131, 0.23770492,
       0.26229508, 0.27868852, 0.31967213, 0.3442623 , 0.36065574,
       0.40163934, 0.40163934, 0.44262295, 0.49180328, 0.53278689,
       0.57377049, 0.6147541 , 0.63934426, 0.6557377 , 0.69672131,
       0.71311475, 0.74590164, 0.7704918 , 0.81967213, 0.8442623 ,
       0.8442623 , 0.86885246, 0.8852459 , 0.90983607, 0.92622951,
       0.92622951, 0.93442623, 0.94262295, 0.95081967, 0.95901639,
       0.96721311, 0.98360656, 0.98360656, 0.98360656, 0.98360656,
       0.98360656, 0.99180328, 0.99180328, 1.        ]), 'tpr': array([0.        , 0.        , 0.        , 0.00847458, 0.00847458,
       0.00847458, 0.00847458, 0.01694915, 0.01694915, 0.01694915,
       0.03389831, 0.04237288, 0.04237288, 0.05932203, 0.07627119,
       0.08474576, 0.101

In [ ]:
np.mean(y_test)

In [14]:
np.mean(y_test, axis=0)

target    0.491667
dtype: float64

In [15]:
np.mean(y_test, axis=1)

1010    1.0
980     1.0
1093    1.0
104     0.0
119     0.0
       ... 
989     1.0
664     1.0
898     1.0
106     0.0
853     1.0
Length: 240, dtype: float64

In [16]:
y_pred

NameError: name 'y_pred' is not defined

In [17]:
a = 1
a

1

In [96]:
b = np.round(1.2323,3)
b

1.232

In [20]:
!pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)


In [141]:
pred_y1_test = RNet(X1_train, y1_train, X1_test, y1_test, 2, 5, 1, 50)

0epoch // loss =2.7901371140615083e-05
10epoch // loss =2.4918226699810475e-06
20epoch // loss =1.5307712146750418e-06
30epoch // loss =6.173209112603217e-05
40epoch // loss =2.924541877291631e-05
tensor([[0.0572],
        [0.0585],
        [0.0590],
        [0.0701],
        [0.0690],
        [0.0598],
        [0.0591],
        [0.0578],
        [0.0721],
        [0.0592],
        [0.0580],
        [0.0586],
        [0.0575],
        [0.0597],
        [0.0584],
        [0.0608],
        [0.0597],
        [0.0638],
        [0.0602],
        [0.0636],
        [0.0665],
        [0.0590],
        [0.0592],
        [0.0583],
        [0.0590],
        [0.0608],
        [0.0596],
        [0.0588],
        [0.0581],
        [0.0589],
        [0.0624],
        [0.0603],
        [0.0601],
        [0.0730],
        [0.0100],
        [0.0644],
        [0.0579],
        [0.0603],
        [0.0645],
        [0.0600],
        [0.0584],
        [0.0667],
        [0.0586],
        [0.0592],
        [0.

AttributeError: 'dict' object has no attribute 'detach'

In [ ]:
print(pred_y1_test)

In [10]:
RNET = torch.load("RNet.pt")
RNET.eval()

RegressionNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=3, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [52]:
CNET = torch.load("CNet.pt")
CNET.eval()

ClassificationNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=4, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [121]:
pred = ANN_model_loader("RNet.pt", X1_test) 
pred2 = ANN_model_loader("CNet.pt", X_test) 

In [123]:
print(pred2)

[[-0.431688  ]
 [-0.51064414]
 [-0.5007413 ]
 [-0.5085681 ]
 [-0.48137718]
 [-0.47569665]
 [-0.478481  ]
 [-0.53478116]
 [-0.64481914]
 [-0.5164831 ]
 [-0.48913002]
 [-0.64923143]
 [-0.624988  ]
 [-0.48717114]
 [-0.49688876]
 [-0.5269459 ]
 [-0.5596838 ]
 [-0.49450058]
 [-0.4982707 ]
 [-0.5240153 ]
 [-0.4544198 ]
 [-0.4870419 ]
 [-0.533968  ]
 [-0.45932218]
 [-0.5463269 ]
 [-0.5105579 ]
 [-0.5900216 ]
 [-0.5267583 ]
 [-0.62464166]
 [-0.5115942 ]
 [-0.47728044]
 [-0.5393333 ]
 [-0.4965279 ]
 [-0.47877356]
 [-0.6391877 ]
 [-0.4844889 ]
 [-0.5136964 ]
 [-0.6335081 ]
 [-0.5255253 ]
 [-0.5122563 ]
 [-0.65844977]
 [-0.51188   ]
 [-0.45833248]
 [-0.49279922]
 [-0.6692928 ]
 [-0.5044303 ]
 [-0.4799586 ]
 [-0.5838595 ]
 [-0.52473974]
 [-0.4903763 ]
 [-0.4688428 ]
 [-0.50462615]
 [-0.5484752 ]
 [-0.519225  ]
 [-0.4866736 ]
 [-0.4671256 ]
 [-0.49169773]
 [-0.5148054 ]
 [-0.58003795]
 [-0.5117135 ]
 [-0.5340092 ]
 [-0.48028278]
 [-0.43794397]
 [-0.50472754]
 [-0.48461336]
 [-0.48448232]
 [-0.58622

In [135]:
X_test = torch.Tensor(X_test.values)
pred_y_test = CNET.predict(X_test)
pred_y_test= pred_y_test['Predarray']
print(pred_y_test)

tensor([[0.1365],
        [0.1475],
        [0.1428],
        [0.1460],
        [0.1430],
        [0.1421],
        [0.1432],
        [0.1492],
        [0.1723],
        [0.1490],
        [0.1479],
        [0.1490],
        [0.1564],
        [0.1437],
        [0.1466],
        [0.1459],
        [0.1458],
        [0.1421],
        [0.1448],
        [0.1443],
        [0.1547],
        [0.1421],
        [0.1421],
        [0.1587],
        [0.1534],
        [0.1437],
        [0.1555],
        [0.1509],
        [0.1654],
        [0.1452],
        [0.1421],
        [0.1512],
        [0.1421],
        [0.1421],
        [0.1639],
        [0.1421],
        [0.1440],
        [0.1512],
        [0.1490],
        [0.1480],
        [0.1583],
        [0.1493],
        [0.1421],
        [0.1430],
        [0.1467],
        [0.1436],
        [0.1421],
        [0.1563],
        [0.1430],
        [0.1421],
        [0.1421],
        [0.1484],
        [0.1472],
        [0.1421],
        [0.1421],
        [0

In [126]:
x = pred_y_test.items()

print(x)

AttributeError: 'Tensor' object has no attribute 'items'

In [137]:
X_test = X_test.numpy()

In [138]:
print(X_test)

[[ 3.80602336e+00  7.63494074e-01 -9.77539539e-01  1.22985256e+00]
 [ 2.88958907e+00 -9.56516191e-02  7.65324593e-01 -9.44870412e-02]
 [-2.81016648e-01  1.17189419e+00  6.46489739e-01  1.38244259e+00]
 [-1.38564634e+00  1.03286839e+00  9.64946330e-01  1.18931937e+00]
 [-3.84974442e-02 -1.02425289e+00 -2.86061287e-01  1.26748371e+00]
 [ 9.19930637e-01  8.03538024e-01  6.98452473e-01  1.89077008e+00]
 [ 1.52581644e+00  1.26883960e+00  1.14634144e+00  1.60621655e+00]
 [ 8.93510342e-01 -7.05286086e-01  1.13732183e+00  1.85925201e-01]
 [-7.45542467e-01  1.20459169e-01  1.77639592e+00 -1.88747442e+00]
 [ 1.36878800e+00  1.86112916e+00  1.96436131e+00  9.19935822e-01]
 [ 2.69998002e+00  5.97405493e-01  1.54226673e+00  6.48979664e-01]
 [-8.96511495e-01 -6.84065670e-02 -1.07213056e+00 -1.85259724e+00]
 [-1.97832420e-01  1.87706971e+00  1.81530488e+00 -4.25111383e-01]
 [ 1.40733492e+00  8.21575403e-01  2.77685165e-01  3.94608140e-01]
 [ 1.38334978e+00 -1.24526672e-01  9.10424232e-01  6.20447576e

In [12]:
z = torch.rand(3, 5, requires_grad=True)

In [13]:
print(z)

tensor([[0.0686, 0.6302, 0.0431, 0.9091, 0.8628],
        [0.4076, 0.3657, 0.6278, 0.1684, 0.8510],
        [0.4725, 0.2610, 0.0672, 0.1658, 0.0351]], requires_grad=True)


In [14]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1211, 0.2123, 0.1180, 0.2806, 0.2679],
        [0.1802, 0.1728, 0.2245, 0.1418, 0.2807],
        [0.2592, 0.2098, 0.1728, 0.1908, 0.1674]], grad_fn=<SoftmaxBackward0>)


In [15]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([2, 2, 0])


In [16]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [17]:
print(y.unsqueeze(1))

tensor([[2],
        [2],
        [0]])


In [97]:
x = np.arange(10).reshape(5,-1)

In [98]:
print(x)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


In [99]:
x = x.reshape(-1)

In [100]:
print(x)

[0 1 2 3 4 5 6 7 8 9]
